In [1]:
from google.cloud import bigquery
import pandas as pd
import plotly.express as px

In [2]:
key_path = '../credentials/dbt_service_account.json'
client = bigquery.Client.from_service_account_json(key_path)

In [3]:
query = """
SELECT
    SUM(CASE WHEN sessions IS NULL THEN 1 ELSE 0 END) AS null_sessions,
    SUM(CASE WHEN purchases IS NULL THEN 1 ELSE 0 END) AS null_purchases,
    SUM(CASE WHEN revenue IS NULL THEN 1 ELSE 0 END) AS null_revenue,
    COUNT(*) AS total_rows
FROM `amiable-dynamo-461319-g1.ga4_demo.fact_ab_cohort`
"""

nulls_df = client.query(query).to_dataframe()
nulls_df

,null_sessions,null_purchases,null_revenue,total_rows
0,0,0,167940,179085


Explore null revenue cases

In [4]:
query_null_revenue = """
SELECT
    sessions,
    purchases,
    COUNT(*) AS users_with_null_revenue
FROM `amiable-dynamo-461319-g1.ga4_demo.fact_ab_cohort`
WHERE revenue IS NULL
GROUP BY sessions, purchases
ORDER BY users_with_null_revenue DESC
"""
null_revenue_exploration_df = client.query(query_null_revenue).to_dataframe()

In [5]:
null_revenue_exploration_df

,sessions,purchases,users_with_null_revenue
0,1,0,140434
1,2,0,18683
2,3,0,4627
3,4,0,1951
4,5,0,959
5,6,0,498
6,7,0,311
7,8,0,184
8,9,0,89
9,10,0,70


In [6]:
query_null_revenue_overview = """
WITH null_rev AS (
    SELECT
        CASE 
            WHEN purchases = 0 THEN '0 purchases'
            ELSE '> purchases'
        END AS purchase_bucket,
    FROM `amiable-dynamo-461319-g1.ga4_demo.fact_ab_cohort`
    WHERE revenue IS NULL
) 

SELECT
    purchase_bucket,
    COUNT(*) AS users,
    ROUND( COUNT(*) / SUM(COUNT(*)) OVER(), 4) AS pct_nulls
FROM null_rev
GROUP BY purchase_bucket
ORDER BY purchase_bucket
    
"""
null_revenue_overview_df = client.query(query_null_revenue_overview).to_dataframe()
null_revenue_overview_df

,purchase_bucket,users,pct_nulls
0,0 purchases,167912,0.9998
1,> purchases,28,0.0002


In [7]:
fig = px.bar(
    null_revenue_overview_df,
    x = 'purchase_bucket',
    y = 'users',
    text = null_revenue_overview_df["pct_nulls"].apply(lambda v: f"{v:.2%}"),
    title='Percentage of Users with Null Revenue by Purchase Bucket', 
    labels = {"pct_nulls":"Percent of NULL-revenue users", "purchase_bucket":""}
)
fig.update_layout(yaxis_tickformat=',.0f', yaxis_title='Number of Users')
fig.show()

In [8]:
fig_pie = px.pie(
    null_revenue_overview_df,
    values='users',
    names='purchase_bucket',
    title='Percentage of Users with Null Revenue by Purchase Bucket',
    labels={"pct_nulls": "Percent of NULL-revenue users", "purchase_bucket": ""}
)
fig_pie.update_traces(texttemplate='%{label}<br>%{percent:.2%}')
fig_pie.show()


Most null revenue cases correspond to the 0 performed purchases. These revenue sitautions should be 0 and not NULL. In other cases, the purchase has occured - further investigation of these cases needed.

#### Explore null revenue and > 0 purchases

In [9]:
non_null_purchases_query = """
SELECT
    sessions, 
    purchases,
    COUNT(*) AS users
FROM `amiable-dynamo-461319-g1.ga4_demo.fact_ab_cohort`
WHERE revenue IS NULL
    AND purchases > 0
GROUP BY sessions, purchases
ORDER BY users DESC    
"""

non_null_purchases_df = client.query(non_null_purchases_query).to_dataframe()
non_null_purchases_df

,sessions,purchases,users
0,2,1,6
1,1,1,5
2,3,1,4
3,4,1,4
4,5,1,4
5,6,1,2
6,4,2,1
7,8,1,1
8,10,1,1


In [10]:
deep_inspect_query = """
SELECT
    user_pseudo_id,
    sessions,
    purchases
FROM `amiable-dynamo-461319-g1.ga4_demo.fact_ab_cohort`
WHERE revenue IS NULL AND purchases > 0
LIMIT 100
"""
deep_inspect_df = client.query(deep_inspect_query).to_dataframe()
deep_inspect_df

,user_pseudo_id,sessions,purchases
0,51270177.6841667595,1,1
1,80598518.3764258754,1,1
2,8571652.3249012608,1,1
3,63037540.6013143839,1,1
4,1396855.2660738879,1,1
5,8015042.5369619169,2,1
6,64087102.2456314175,2,1
7,38693124.8629644455,2,1
8,5568834.0198043803,2,1
9,48751682.0323834062,2,1


Pass the user pseudo IDs as a query parameter for stg_events

In [13]:
user_ids = list(deep_inspect_df['user_pseudo_id'].unique())
user_ids

['51270177.6841667595',
 '80598518.3764258754',
 '8571652.3249012608',
 '63037540.6013143839',
 '1396855.2660738879',
 '8015042.5369619169',
 '64087102.2456314175',
 '38693124.8629644455',
 '5568834.0198043803',
 '48751682.0323834062',
 '60304773.1030019476',
 '3398472.6191412239',
 '17270185.4245519703',
 '49394451.7466103154',
 '70482486.1697864148',
 '70947919.9290448682',
 '73111349.9152668013',
 '72491659.8341700565',
 '36921495.1660009263',
 '56920896.3476155826',
 '55601747.6727525025',
 '13429550.1174612381',
 '91541812.0825246466',
 '9202815833.2374673362',
 '34497430.3741988669',
 '4921679.6988965991',
 '47629981.2636166951',
 '3398888.3003961025']

In [30]:
job_config = bigquery.QueryJobConfig(
    query_parameters=[
        bigquery.ArrayQueryParameter("user_ids", "STRING", user_ids)
    ]
)

query_ids = """
SELECT *
FROM `amiable-dynamo-461319-g1.ga4_demo.stg_events`
WHERE user_pseudo_id IN UNNEST(@user_ids)
ORDER BY user_pseudo_id, event_ts
"""
ids_df = client.query(query_ids, job_config=job_config).to_dataframe()
ids_df

,event_ts,event_date,event_name,user_pseudo_id,platform,country,region,traffic_name,traffic_medium,traffic_source,ga_session_id,event_params,user_properties,items
0,2020-11-26 05:08:05.945465+00:00,2020-11-26,session_start,13429550.1174612381,WEB,India,Tamil Nadu,(organic),organic,google,9742101714,"[{'key': 'page_location', 'value': {'string_va...",[],[]
1,2020-11-26 05:08:05.945465+00:00,2020-11-26,first_visit,13429550.1174612381,WEB,India,Tamil Nadu,(organic),organic,google,9742101714,"[{'key': 'engaged_session_event', 'value': {'s...",[],[]
2,2020-11-26 05:08:05.945465+00:00,2020-11-26,page_view,13429550.1174612381,WEB,India,Tamil Nadu,(organic),organic,google,9742101714,"[{'key': 'clean_event', 'value': {'string_valu...",[],[]
3,2020-11-26 05:08:37.529970+00:00,2020-11-26,user_engagement,13429550.1174612381,WEB,India,Tamil Nadu,(organic),organic,google,9742101714,"[{'key': 'all_data', 'value': {'string_value':...",[],[]
4,2020-11-26 05:08:44.690391+00:00,2020-11-26,page_view,13429550.1174612381,WEB,India,Tamil Nadu,(organic),organic,google,9742101714,"[{'key': 'all_data', 'value': {'string_value':...",[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4785,2020-12-22 23:35:26.709068+00:00,2020-12-22,select_item,9202815833.2374673362,WEB,United States,North Carolina,(direct),(none),(direct),5928621231,"[{'key': 'session_engaged', 'value': {'string_...",[],"[{'item_id': 'GGOEGXXX1100', 'item_name': 'Goo..."
4786,2020-12-22 23:35:26.709068+00:00,2020-12-22,view_item,9202815833.2374673362,WEB,United States,North Carolina,(direct),(none),(direct),5928621231,"[{'key': 'ga_session_number', 'value': {'strin...",[],"[{'item_id': 'GGOEGXXX1100', 'item_name': 'Goo..."
4787,2020-12-22 23:35:35.089200+00:00,2020-12-22,view_item,9202815833.2374673362,WEB,United States,North Carolina,(direct),(none),(direct),5928621231,"[{'key': 'engagement_time_msec', 'value': {'st...",[],[]
4788,2020-12-22 23:35:35.089200+00:00,2020-12-22,page_view,9202815833.2374673362,WEB,United States,North Carolina,(direct),(none),(direct),5928621231,"[{'key': 'debug_mode', 'value': {'string_value...",[],[]


In [31]:
# Sanity check: number of unique users in the ids_df
ids_df['user_pseudo_id'].unique().shape[0]  

28

In [50]:
# Drop rows with empty items cell
items_df = ids_df[ids_df['items'].apply(lambda x: len(x) > 0)]
items_df

,event_ts,event_date,event_name,user_pseudo_id,platform,country,region,traffic_name,traffic_medium,traffic_source,ga_session_id,event_params,user_properties,items
6,2020-11-26 05:10:48.189592+00:00,2020-11-26,view_item,13429550.1174612381,WEB,India,Tamil Nadu,(organic),organic,google,9742101714,"[{'key': 'engagement_time_msec', 'value': {'st...",[],"[{'item_id': 'GGOEAFKA087599', 'item_name': 'A..."
12,2020-11-26 05:15:32.132472+00:00,2020-11-26,view_item,13429550.1174612381,WEB,India,Tamil Nadu,(organic),organic,google,9742101714,"[{'key': 'engagement_time_msec', 'value': {'st...",[],"[{'item_id': 'GGOEAOCB092899', 'item_name': 'A..."
23,2020-11-26 05:19:53.233146+00:00,2020-11-26,view_item,13429550.1174612381,WEB,India,Tamil Nadu,(organic),organic,google,9742101714,"[{'key': 'ga_session_number', 'value': {'strin...",[],"[{'item_id': 'GGCOGXXX1609', 'item_name': 'Goo..."
32,2020-11-26 05:23:44.244454+00:00,2020-11-26,view_item,13429550.1174612381,WEB,India,Tamil Nadu,(organic),organic,google,9742101714,"[{'key': 'page_title', 'value': {'string_value...",[],"[{'item_id': 'GGOEGAPH161899', 'item_name': 'G..."
41,2020-11-26 05:26:07.136173+00:00,2020-11-26,view_item,13429550.1174612381,WEB,India,Tamil Nadu,(organic),organic,google,9742101714,"[{'key': 'ga_session_number', 'value': {'strin...",[],"[{'item_id': 'GGCOGDWH161099', 'item_name': 'G..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4763,2020-12-15 22:28:34.483351+00:00,2020-12-15,purchase,9202815833.2374673362,WEB,United States,North Carolina,(data deleted),(data deleted),(data deleted),6050979833,"[{'key': 'engagement_time_msec', 'value': {'st...",[],"[{'item_id': '(not set)', 'item_name': '(not s..."
4776,2020-12-22 02:53:03.895272+00:00,2020-12-22,view_item,9202815833.2374673362,WEB,United States,North Carolina,(data deleted),(data deleted),(data deleted),3675596011,"[{'key': 'page_location', 'value': {'string_va...",[],"[{'item_id': 'GGOEGHPB120910', 'item_name': 'G..."
4777,2020-12-22 02:53:03.895272+00:00,2020-12-22,select_item,9202815833.2374673362,WEB,United States,North Carolina,(data deleted),(data deleted),(data deleted),3675596011,"[{'key': 'page_referrer', 'value': {'string_va...",[],"[{'item_id': 'GGOEGHPB120910', 'item_name': 'G..."
4785,2020-12-22 23:35:26.709068+00:00,2020-12-22,select_item,9202815833.2374673362,WEB,United States,North Carolina,(direct),(none),(direct),5928621231,"[{'key': 'session_engaged', 'value': {'string_...",[],"[{'item_id': 'GGOEGXXX1100', 'item_name': 'Goo..."


In [51]:
# Explode the items column so each item in the list gets its own row
items_df_exploded = items_df.explode('items').reset_index(drop=True)

# Expand the dictionary in the 'items' column into separate columns
items_expanded = items_df_exploded['items'].apply(pd.Series)

# Join the expanded columns back to the exploded dataframe (excluding the original 'items' column)
items_df_expanded = pd.concat([items_df_exploded.drop(columns=['items']), items_expanded], axis=1)

items_df_expanded.head()

,event_ts,event_date,event_name,user_pseudo_id,platform,country,region,traffic_name,traffic_medium,traffic_source,...,coupon,affiliation,location_id,item_list_id,item_list_name,item_list_index,promotion_id,promotion_name,creative_name,creative_slot
0,2020-11-26 05:10:48.189592+00:00,2020-11-26,view_item,13429550.1174612381,WEB,India,Tamil Nadu,(organic),organic,google,...,(not set),(not set),(not set),(not set),Not available in demo dataset,11,(not set),(not set),(not set),(not set)
1,2020-11-26 05:10:48.189592+00:00,2020-11-26,view_item,13429550.1174612381,WEB,India,Tamil Nadu,(organic),organic,google,...,(not set),(not set),(not set),(not set),Not available in demo dataset,5,(not set),(not set),(not set),(not set)
2,2020-11-26 05:10:48.189592+00:00,2020-11-26,view_item,13429550.1174612381,WEB,India,Tamil Nadu,(organic),organic,google,...,(not set),(not set),(not set),(not set),Not available in demo dataset,9,(not set),(not set),(not set),(not set)
3,2020-11-26 05:10:48.189592+00:00,2020-11-26,view_item,13429550.1174612381,WEB,India,Tamil Nadu,(organic),organic,google,...,(not set),(not set),(not set),(not set),Not available in demo dataset,8,(not set),(not set),(not set),(not set)
4,2020-11-26 05:10:48.189592+00:00,2020-11-26,view_item,13429550.1174612381,WEB,India,Tamil Nadu,(organic),organic,google,...,(not set),(not set),(not set),(not set),Not available in demo dataset,3,(not set),(not set),(not set),(not set)


In [54]:
items_df_expanded.drop(columns = ['item_name', 'item_brand',
       'item_variant', 'item_category', 'item_category2',
       'item_category3', 'item_category4', 'item_category5', 'affiliation', 'location_id', 'item_list_id',
       'item_list_name', 'item_list_index',
       'promotion_name', 'creative_name', 'creative_slot', 'platform',
       'country', 'region', 'traffic_name', 'traffic_medium', 'traffic_source', 'event_params', 
       'user_properties'], inplace=True)

In [57]:
items_df_expanded.head()

,event_ts,event_date,event_name,user_pseudo_id,ga_session_id,item_id,price_in_usd,price,quantity,item_revenue_in_usd,item_revenue,item_refund_in_usd,item_refund,coupon,promotion_id
0,2020-11-26 05:10:48.189592+00:00,2020-11-26,view_item,13429550.1174612381,9742101714,GGOEAFKA087599,None,3.0,None,None,None,None,None,(not set),(not set)
1,2020-11-26 05:10:48.189592+00:00,2020-11-26,view_item,13429550.1174612381,9742101714,GGOEAFBA115499,None,40.0,None,None,None,None,None,(not set),(not set)
2,2020-11-26 05:10:48.189592+00:00,2020-11-26,view_item,13429550.1174612381,9742101714,GGOEGOCL092999,None,13.0,None,None,None,None,None,(not set),(not set)
3,2020-11-26 05:10:48.189592+00:00,2020-11-26,view_item,13429550.1174612381,9742101714,GGOEAOCB092899,None,13.0,None,None,None,None,None,(not set),(not set)
4,2020-11-26 05:10:48.189592+00:00,2020-11-26,view_item,13429550.1174612381,9742101714,GGOEGXXX0905,None,22.0,None,None,None,None,None,(not set),(not set)


In [58]:
print(
    f"Unique price in usd: {items_df_expanded['price_in_usd'].unique()}\n"
    f"Unique quantity: {items_df_expanded['quantity'].unique()}\n"
    f"Unique item_revenue: {items_df_expanded['item_revenue'].unique()}"
)

Unique price in usd: [None]
Unique quantity: [None]
Unique item_revenue: [None]


It seems that the null revenue cases with > 0 purchases lack values for price in USD, quantity and item revenue, influencing the revenue being null.

#### Conclusion
There are two cases where revenue is null and the following treatments will be applied to the data:
- revenue null & purchases = 0 &rarr; revenue = 0
- revenue null & purchases > 0 &rarr; 28 cases where "price in usd" and "quanity" missing. These cases will be removed.